<a href="https://colab.research.google.com/github/skywalker0803r/RL/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 1. 安裝必要的庫 (使用 Gymnasium, 這是 OpenAI Gym 的維護版本)
#!pip install gymnasium stable-baselines3

In [8]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import random
import time

# --- 常數定義 ---
CONVEYOR_LENGTH = 10.0  # 輸送帶總長度 (米)
ARM_RANGE = 2.0         # 機械手臂的工作範圍 (X, Y 軸)
ARM_Z = 1.5             # 機械手臂的固定高度 (Z 軸)
V_MAX = 0.5             # 輸送帶最大速度 (米/秒)
TIME_STEP = 0.1         # 模擬時間步長 (秒)
MAX_PRODUCTS = 5        # 輸送帶上最多產品數量

class ConveyorAssemblyEnv(gym.Env):
    """
    輸送帶組裝系統的強化學習環境 (連續控制簡化版)
    Agent: 機械手臂控制器 + 輸送帶速度控制器
    """
    metadata = {'render_modes': ['human'], 'render_fps': 30}

    def __init__(self, render_mode=None):
        super(ConveyorAssemblyEnv, self).__init__()

        # --- 狀態空間 (Observation Space) ---
        # 狀態向量包含：
        # 1. 產品 1 - 5 的位置 (x_1, x_2, ..., x_5), 0 表示無產品
        # 2. 機械手臂的當前位置 (arm_x, arm_y)
        # 3. 系統時鐘 (current_time)
        # 4. 已完成組裝的產品計數 (shipped_count)
        self.observation_space = spaces.Box(
            # 注意: np.inf 和 MAX_PRODUCTS * 10 由於被轉換為 float32，會有精度警告，但通常不影響 RL 訓練
            low=np.array([0.0] * MAX_PRODUCTS + [-ARM_RANGE, -ARM_RANGE, 0.0, 0], dtype=np.float32),
            high=np.array([CONVEYOR_LENGTH] * MAX_PRODUCTS + [ARM_RANGE, ARM_RANGE, np.inf, MAX_PRODUCTS * 10], dtype=np.float32),
            dtype=np.float32 # 確保 Space 的 dtype 是 float32
        )

        # --- 動作空間 (Action Space) ---
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(4,), dtype=np.float32)

        self.products = []
        self.arm_pos = np.array([0.0, 0.0], dtype=np.float32) # 確保內部狀態也是 float32
        self.current_time = 0.0
        self.shipped_count = 0
        self.assembly_point = CONVEYOR_LENGTH / 2

    def _get_obs(self):
        # 將產品位置和手臂位置組合成狀態向量
        product_positions = np.zeros(MAX_PRODUCTS)
        for i, product in enumerate(self.products):
            if i < MAX_PRODUCTS:
                product_positions[i] = product['x']

        observation_vector = np.concatenate([
            product_positions,
            self.arm_pos,
            np.array([self.current_time, self.shipped_count])
        ])

        # 關鍵修復點: 將輸出強制轉換為 np.float32
        return observation_vector.astype(np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        self.current_time = 0.0
        self.shipped_count = 0
        self.arm_pos = np.array([0.0, 0.0], dtype=np.float32)

        # 初始狀態: 放置 2 個產品在輸送帶起點附近
        self.products = [
            {'id': 1, 'x': 0.1, 'status': 'waiting_assembly'},
            {'id': 2, 'x': 0.5, 'status': 'waiting_assembly'}
        ]

        # 確保產品數量不超過最大值
        while len(self.products) < MAX_PRODUCTS:
            # 【關鍵修正】: 佔位符位置必須在 [0.0, 10.0] 範圍內
            self.products.append({'id': len(self.products) + 1, 'x': 0.0, 'status': 'shipped_placeholder'})

        observation = self._get_obs()
        info = {}
        return observation, info

    def step(self, action):
        # 1. 解析動作
        v1_factor, v3_factor, arm_x_target_norm, arm_y_target_norm = action

        # 將標準化動作轉換為實際控制量
        v1 = V_MAX * np.clip(v1_factor, 0, 1)
        v3 = V_MAX * np.clip(v3_factor, 0, 1)
        arm_target = np.array([ARM_RANGE * arm_x_target_norm, ARM_RANGE * arm_y_target_norm], dtype=np.float32)

        # 2. 模擬機械手臂移動
        arm_velocity = (arm_target - self.arm_pos) * 0.5
        self.arm_pos += arm_velocity * TIME_STEP
        self.arm_pos = np.clip(self.arm_pos, -ARM_RANGE, ARM_RANGE).astype(np.float32) # 保持為 float32

        # 3. 模擬輸送帶和產品移動 (其餘邏輯不變)
        for product in self.products:
            if product['status'] == 'waiting_assembly':
                product['x'] += v1 * TIME_STEP
            elif product['status'] == 'waiting_shipment':
                product['x'] += v3 * TIME_STEP

        # 4. 執行組裝/出貨邏輯 (Reward Calculation的核心)
        reward = 0.0

        # ... (中略，邏輯保持不變) ...
        for product in self.products:
            if product['status'] == 'waiting_assembly' and product['x'] > 0 and product['x'] < CONVEYOR_LENGTH:
                arm_near_product = np.linalg.norm(self.arm_pos - [product['x'], 0]) < 0.2
                if arm_near_product:
                    product['status'] = 'assembled'
                    reward += 1.0

            elif product['status'] == 'assembled':
                 arm_near_product = np.linalg.norm(self.arm_pos - [product['x'], 0]) < 0.2
                 arm_near_shipment_start = np.linalg.norm(self.arm_pos - [CONVEYOR_LENGTH, 0]) < 0.2

                 if arm_near_product:
                     product['status'] = 'gripped'
                 elif product['status'] == 'gripped' and arm_near_shipment_start:
                     product['status'] = 'waiting_shipment'
                     product['x'] = 0.1
                     reward += 1.0

            elif product['status'] == 'waiting_shipment':
                if product['x'] >= CONVEYOR_LENGTH:
                    product['status'] = 'shipped'
                    self.shipped_count += 1
                    reward += 10.0

                    new_id = max(p['id'] for p in self.products) + 1
                    self.products.append({'id': new_id, 'x': 0.1, 'status': 'waiting_assembly'})
                    self.products = [p for p in self.products if p['status'] != 'shipped'][:MAX_PRODUCTS]


        # 5. 複合懲罰項
        reward -= 0.1 * (v1**2 + v3**2)
        reward -= 0.05 * TIME_STEP

        if v1 > 0.4 or v3 > 0.4:
             reward -= 0.5 * (v1 + v3)

        # 6. 更新時間和終止條件
        self.current_time += TIME_STEP

        terminated = self.current_time >= 300.0
        truncated = False

        observation = self._get_obs() # 返回的 observation 已經是 float32

        # 【💡 關鍵修復點】: 將 reward 轉換為標準 Python float
        # 使用 float() 函數來確保類型正確
        python_float_reward = float(reward)

        return observation, python_float_reward, terminated, truncated, {}

# 檢查環境是否符合標準 (在 Colab 中運行，確認環境定義無誤)
from stable_baselines3.common.env_checker import check_env
env = ConveyorAssemblyEnv()
try:
    check_env(env, warn=True)
    print("✅ 環境檢查通過 (Env Check Passed)")
except Exception as e:
    print(f"❌ 環境檢查失敗: {e}")

✅ 環境檢查通過 (Env Check Passed)


In [9]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback

# 設置 TensorBoard log 路徑
LOG_DIR = "./assembly_ppo_tensorboard/"
os.makedirs(LOG_DIR, exist_ok=True)

class CustomCallback(BaseCallback):
    """
    自定義回調函數，用於在訓練期間報告進度
    """
    def __init__(self, verbose=0):
        super(CustomCallback, self).__init__(verbose)
        self.check_freq = 5000  # 每 5000 步保存一次
        self.save_path = './best_model'
        os.makedirs(self.save_path, exist_ok=True)
        self.best_mean_reward = -np.inf

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            # 簡化評估: 使用訓練過程中的平均獎勵作為指標
            mean_reward = np.mean(self.model.ep_info_buffer[-10:]) if self.model.ep_info_buffer else -np.inf

            if mean_reward > self.best_mean_reward:
                self.best_mean_reward = mean_reward
                if self.verbose > 0:
                    print(f"Saving new best model at step {self.num_timesteps} with mean reward {self.best_mean_reward:.2f}")
                self.model.save(os.path.join(self.save_path, 'best_model.zip'))
        return True

# 1. 初始化環境
env = ConveyorAssemblyEnv()
# 將環境包裝在 DummyVecEnv 中以兼容 Stable Baselines3
env = DummyVecEnv([lambda: env])

# 2. 定義 PPO 模型
# MlpPolicy: 使用多層感知器 (MLP) 作為策略網路
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=3e-4,
    gamma=0.99,            # 折扣因子 (重視長期獎勵)
    n_steps=2048,          # 每次更新採集的步數
    ent_coef=0.01,         # 熵係數 (鼓勵探索)
    tensorboard_log=LOG_DIR
)

print("--- 開始訓練 PPO Agent ---")

# 3. 訓練模型
TIMESTEPS = 1000 # 總訓練時間步
callback = CustomCallback(verbose=1)

model.learn(
    total_timesteps=TIMESTEPS,
    callback=callback
)

print("--- 訓練完成 ---")
model.save("final_assembly_ppo_model.zip")

Using cuda device
--- 開始訓練 PPO Agent ---
Logging to ./assembly_ppo_tensorboard/PPO_2
-----------------------------
| time/              |      |
|    fps             | 921  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
--- 訓練完成 ---


In [13]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import numpy as np
import os
# 假設 V_MAX 在之前已經定義為 0.5

# 載入最佳模型 (或最終模型)
# 確保 'best_model/best_model.zip' 或 'final_assembly_ppo_model.zip' 存在
try:
    # 嘗試載入訓練中保存的最佳模型
    model = PPO.load("./best_model/best_model.zip", env=env)
except Exception:
    # 如果不存在，則載入最終模型
    print("Warning: Best model not found. Loading final model.")
    model = PPO.load("final_assembly_ppo_model.zip", env=env)


# 1. 評估模型性能
print("\n--- 策略評估 ---")
# evaluate_policy 會自動處理 DummyVecEnv 的 step 返回值
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"平均回合獎勵: {mean_reward:.2f} +/- {std_reward:.2f}")


# 2. 策略運行演示 (單次回合)
print("\n--- 運行最佳策略 ---")

# 正確解包 DummyVecEnv 的 reset()
# obs 是一個長度為 1 的陣列，包含第一個環境的狀態
obs = env.reset()

done = False
total_steps = 0
V_MAX = 0.5 # 重新定義 V_MAX 以防丟失

while not done:
    # 決定性動作 (deterministic=True) 用於部署
    action, _ = model.predict(obs, deterministic=True)

    # DummyVecEnv 的 step() 返回五個陣列 (obs, reward, terminated, truncated, info)
    obs, reward, terminated, truncated = env.step(action)

    # 只要任何一個環境結束 (terminated 或 truncated)，就設為 True
    # terminated 和 truncated 都是長度為 1 的布林陣列
    done = terminated[0] or truncated[0]

    total_steps += 1

    # 由於 obs 是一個陣列，我們需要取第一個元素 [0] 來獲取實際的狀態向量
    # current_shipped 是狀態向量的最後一個元素
    current_shipped = obs[0][-1]

    if total_steps % 50 == 0:
        # 觀察關鍵狀態 (obs, action, reward 都是陣列，取 [0] 獲取第一個環境的值)
        current_time = obs[0][-2]

        # action 是一個陣列，包含第一個環境的動作
        v1_ctrl = action[0][0] * V_MAX
        v3_ctrl = action[0][1] * V_MAX

        arm_x = obs[0][-4]
        arm_y = obs[0][-3]

        print(f"Time: {current_time:.1f}s | Shipped: {current_shipped:.0f} | R: {reward[0]:.2f}")
        print(f"  V1: {v1_ctrl:.2f} m/s, V3: {v3_ctrl:.2f} m/s | Arm Pos: ({arm_x:.2f}, {arm_y:.2f})")

# 最終結果
final_time = obs[0][-2]
final_shipped = obs[0][-1]
throughput = final_shipped / (final_time / 3600.0) if final_time > 0 else 0

print(f"\n✅ 模擬完成！ 總時長: {final_time:.1f}s")
print(f"總出貨量: {final_shipped:.0f} 個")
print(f"換算每小時出貨量 (Throughput): {throughput:.2f} 個/小時")


--- 策略評估 ---
平均回合獎勵: -14.00 +/- 0.00

--- 運行最佳策略 ---

✅ 模擬完成！ 總時長: 0.1s
總出貨量: 0 個
換算每小時出貨量 (Throughput): 0.00 個/小時
